In [ ]:
#Checking if GPU is running or not

!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
!pip install datasets transformers[sentencepiece] sacrebleu -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 41.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.


In [ ]:
import os
import sys
import transformers
import tensorflow as tf
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import AdamWeightDecay
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM

In [ ]:
#model_checkpoint = "Helsinki-NLP/opus-mt-en-hi"

In [ ]:
model_checkpoint  = "Helsinki-NLP/opus-mt-en-mr"

# The Dataset

Source: https://huggingface.co/datasets/cfilt/iitb-english-hindi

In [ ]:
raw_datasets = load_dataset("SRDdev/Marathi-English")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/3137 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/785 [00:00<?, ? examples/s]

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 3137
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 785
    })
})

In [ ]:
raw_datasets['train'][1]

{'translation': {'en': 'Anything else', 'mr': 'अजुन काही'}}

#Preprocessing the data

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/813k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/1.17M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
tokenizer("Hello, this is a sentence!")

{'input_ids': [50289, 3, 67, 17, 12, 6913, 70, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
tokenizer(["Hello, this is a sentence!", "This is another sentence."])

{'input_ids': [[50289, 3, 67, 17, 12, 6913, 70, 0], [235, 17, 374, 6913, 2, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}

In [ ]:
with tokenizer.as_target_tokenizer():
    print(tokenizer(["अजुन काही"]))

{'input_ids': [[31205, 103, 0]], 'attention_mask': [[1, 1, 1]]}


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
max_input_length = 128
max_target_length = 128

source_lang = "en"
target_lang = "mr"


def preprocess_function(examples):
    inputs = [ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
preprocess_function(raw_datasets["train"][:2])

{'input_ids': [[11902, 1500, 7, 24, 1071, 2, 0], [32665, 1854, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1]], 'labels': [[789, 1215, 4500, 388, 206, 0], [31205, 103, 0]]}

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/3137 [00:00<?, ? examples/s]

Map:   0%|          | 0/785 [00:00<?, ? examples/s]

In [ ]:
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

tf_model.h5:   0%|          | 0.00/306M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at Helsinki-NLP/opus-mt-en-mr.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [ ]:
batch_size = 16
learning_rate = 2e-5
weight_decay = 0.01
num_train_epochs = 1

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

In [ ]:
generation_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf", pad_to_multiple_of=128)

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf", pad_to_multiple_of=2)


In [ ]:
train_dataset = model.prepare_tf_dataset(
    tokenized_datasets["test"],
    batch_size=batch_size,
    shuffle=True,
    collate_fn=data_collator,
)


In [ ]:
print(tokenized_datasets.keys())


dict_keys(['train', 'test'])


In [ ]:
validation_dataset = model.prepare_tf_dataset(
    tokenized_datasets["test"],
    batch_size=batch_size,
    shuffle=False,
    collate_fn=data_collator,
)


In [ ]:
#validation_dataset = model.prepare_tf_dataset(
 #   tokenized_datasets["validation"],
  #  batch_size=batch_size,
  #  shuffle=False,
  #  collate_fn=data_collator,
#)

In [ ]:
from transformers import TFAutoModelForSeq2SeqLM

# Load the Marian model for sequence-to-sequence tasks
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

# Set up data collator for generation
generation_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf", pad_to_multiple_of=128)

# Prepare the generation dataset
generation_dataset = model.prepare_tf_dataset(
    tokenized_datasets["test"],  # Use the test split for generation
    batch_size=8,
    shuffle=False,
    collate_fn=generation_data_collator,
)


All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at Helsinki-NLP/opus-mt-en-mr.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


In [ ]:
#generation_dataset = model.prepare_tf_dataset(
 #   tokenized_datasets["validation"],
   # batch_size=8,
    #shuffle=False,
   # collate_fn=generation_data_collator,
#)

In [ ]:
optimizer = AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=weight_decay)
model.compile(optimizer=optimizer)

In [ ]:
model.fit(train_dataset, validation_data=validation_dataset, epochs=100)

Epoch 1/100
49/49 [==============================] - 45s 328ms/step - loss: 4.1307 - val_loss: 3.4044
Epoch 2/100
49/49 [==============================] - 9s 179ms/step - loss: 3.4744 - val_loss: 2.9399
Epoch 3/100
49/49 [==============================] - 8s 160ms/step - loss: 3.0899 - val_loss: 2.5804
Epoch 4/100
49/49 [==============================] - 8s 164ms/step - loss: 2.7558 - val_loss: 2.2805
Epoch 5/100
49/49 [==============================] - 8s 173ms/step - loss: 2.4531 - val_loss: 2.0282
Epoch 6/100
49/49 [==============================] - 9s 174ms/step - loss: 2.2626 - val_loss: 1.8180
Epoch 7/100
49/49 [==============================] - 9s 181ms/step - loss: 2.0362 - val_loss: 1.6359
Epoch 8/100
49/49 [==============================] - 8s 160ms/step - loss: 1.8935 - val_loss: 1.4878
Epoch 9/100
49/49 [==============================] - 9s 175ms/step - loss: 1.7520 - val_loss: 1.3460
Epoch 10/100
49/49 [==============================] - 8s 173ms/step - loss: 1.5939 - val_l

In [ ]:
model.save_pretrained("tf_model/")

# Model Testing

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = TFAutoModelForSeq2SeqLM.from_pretrained("tf_model/")

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at tf_model/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


In [ ]:
input_text  = input()

tokenized = tokenizer([input_text], return_tensors='np')
out = model.generate(**tokenized, max_length=128)
print(out)

how are you
tf.Tensor([[61673   151     0 61673 61673 61673]], shape=(1, 6), dtype=int32)


In [ ]:
with tokenizer.as_target_tokenizer():
    print(tokenizer.decode(out[0], skip_special_tokens=True))

तू


In [ ]:
input_text  = input()

tokenized = tokenizer([input_text], return_tensors='np')
out = model.generate(**tokenized, max_length=128)
print(out)

idiot
tf.Tensor([[61673  2473     0]], shape=(1, 3), dtype=int32)


In [ ]:
with tokenizer.as_target_tokenizer():
    print(tokenizer.decode(out[0], skip_special_tokens=True))

मूर्ख


In [ ]:
input_text  = input()

tokenized = tokenizer([input_text], return_tensors='np')
out = model.generate(**tokenized, max_length=128)
print(out)

how
tf.Tensor([[61673   680     0]], shape=(1, 3), dtype=int32)


In [ ]:
with tokenizer.as_target_tokenizer():
    print(tokenizer.decode(out[0], skip_special_tokens=True))

कसा


In [ ]:
input_text  = input()

tokenized = tokenizer([input_text], return_tensors='np')
out = model.generate(**tokenized, max_length=128)
print(out)

space
tf.Tensor([[61673  1633     0 61673 61673 61673]], shape=(1, 6), dtype=int32)


In [ ]:
with tokenizer.as_target_tokenizer():
    print(tokenizer.decode(out[0], skip_special_tokens=True))

जागा
